<a href="https://colab.research.google.com/github/GiulioCarloVenturi/ForexFactory/blob/main/ForexFactory_Webscraping_NoOverlap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


## ForexFactory: Webscraping of Announcements and Removal of Overlapping Events

# Scraping of Event Records
Please read the code ForexFactory_Webscapring.ipynb for further details about the scarping procedure.

In [ ]:
start_year = 2024
start_month = 2
start_day = 1
end_year = 2024
end_month = 2
end_day = 12
timezone_UStoITA = 1

In [ ]:
!pip install beautifulsoup4
!pip install cloudscraper
import cloudscraper
from bs4 import BeautifulSoup
import datetime as dt
from datetime import *
import pandas as pd
import pytz

In [ ]:
def url_forexfactory(data):
    month = data.strftime("%b")
    day = data.strftime("%d")
    year = data.year
    url_ff = f"calendar?day={month}{day}.{year}"
    return url_ff

def date_range(start_date, end_date):
    date_list = []
    current_date = start_date
    while current_date <= end_date:
        date_list.append(current_date)
        current_date += dt.timedelta(days=1)
    return date_list

event_date = list()
event_time = list()
currency = list()
forecast = list()
previous = list()
actual = list()
impact = list()
event = list()
n_event = list()
start_date = dt.date(start_year,start_month,start_day)
end_date = dt.date(end_year,end_month,end_day)
date_list = date_range(start_date, end_date)
for j in date_list:
  date_url = url_forexfactory(j)
  scraper = cloudscraper.create_scraper()
  url = 'https://www.forexfactory.com/' + date_url
  page = scraper.get(url).text
  soup = BeautifulSoup(page, 'html.parser')
  table = soup.find('table', class_ = 'calendar__table')
  n_event = len(table.find_all('td', class_ = 'calendar__event'))

  for i in range(n_event):
    event_date.append(j.strftime("%d/%m/%Y"))
    currency.append(table.find_all('td', class_ = 'calendar__currency')[i].text.strip())
    forecast.append(table.find_all('td', class_ = 'calendar__forecast')[i].text.strip())
    previous.append(table.find_all('td', class_ = 'calendar__previous')[i].text.strip())
    actual.append(table.find_all('td', class_ = 'calendar__actual')[i].text.strip())
    event.append(table.find_all('td', class_ = 'calendar__event')[i].text.strip())
    impact.append(table.find_all('td', class_ = 'calendar__impact')[i].find_next('span')['class'][1])
    event_time.append(table.find_all('td', class_ = 'calendar__time')[i].text.strip())
  print(j)

for i in range(len(impact)):
  if impact[i][16:len(impact[i])] == 'red':
    impact[i] = 'Red'
  elif impact[i][16:len(impact[i])] == 'ora':
    impact[i] = 'Orange'
  elif impact[i][16:len(impact[i])] == 'yel':
    impact[i] = 'Yellow'
  elif impact[i][16:len(impact[i])] == 'gra':
    impact[i] = 'Gray'

for i in range(len(event_time)):
  if event_time[i][len(event_time[i])-2:len(event_time[i])] == 'am' or event_time[i][len(event_time[i])-2:len(event_time[i])] == 'pm':
    event_time[i] = datetime.strftime(datetime.strptime(event_time[i], "%I:%M%p"), "%H:%M")
  if event_time[i] == '':
    event_time[i] = event_time[i-1]
  if ((event_time[i] == 'All Day')  and (actual[i] != '')):
    event_time[i] = event_time [i-1]

if timezone_UStoITA == 1:
  for i in range(len(event_time)):
   if event_time[i][2] == ':':
     date_str = f"{event_date[i]} {event_time[i]}"
     date_obj = datetime.strptime(date_str, '%d/%m/%Y %H:%M')
     timestamp_ny = datetime(date_obj.year, date_obj.month, date_obj.day, date_obj.hour, date_obj.minute, tzinfo=pytz.timezone('America/New_York'))
     timestamp_rome = timestamp_ny.astimezone(pytz.timezone('Europe/Rome'))+timedelta(minutes=4)
     event_date[i] = timestamp_rome.strftime("%d/%m/%Y")
     event_time[i] = timestamp_rome.strftime("%H:%M")

ff_data = {'Date': event_date, 'Time': event_time, 'Event': event, 'Currency': currency, 'Impact': impact, 'Actual': actual, 'Forecast': forecast, 'Previous': previous}
ff_dataframe = pd.DataFrame(data=ff_data)

# Detection and Removal of Overlapping Events

Filter announcement records to retrieve events in US and EA with Red and Orange impact.

Build a (t-15',t+15') window around announcementsevents.


In [ ]:
data3 = ff_dataframe.query('(Currency == "USD" or Currency == "EUR") and (Impact == "Red" or Impact == "Orange")', inplace=False)
date_obj = list()
window_l = list()
window_r = list()

for i in data3.index:
  date_str = f"{data3.Date[i]} {data3.Time[i]}"
  date_obj.append(datetime.strptime(date_str, '%d/%m/%Y %H:%M'))
  window_l.append(date_obj[-1] - timedelta(minutes=15))
  window_r.append(date_obj[-1] + timedelta(minutes=15))

d_frame = {'Left': window_l, 'Date': date_obj, 'Right': window_r,
           'Event': data3.Event, 'Currency': data3.Currency,
           'Impact': data3.Impact, 'Actual': data3.Actual,
           'Forecast': data3.Forecast, 'Previous': data3.Previous}
dd_frame = pd.DataFrame(data=d_frame)
dd_frame

Detect and remove announcements in different regions, whose time windows intersect.

E.g. Event X in EA at 14:15 and Event Y in US at 14:30: both are removed

In [ ]:
rem_event = list()
j_event = list()
i_event = list()

for j in data3.index:
#  print(j)
  for i in data3.index:
    if ((dd_frame.Left[j] <= dd_frame.Left[i] <= dd_frame.Right[j]) and (dd_frame.Currency[i] != dd_frame.Currency[j])):
      rem_event.append(i)
      i_event.append(i)
#  print(rem_event)
  if rem_event:
    j_event.append(j)
  rem_event.clear()

index_rem = set(i_event+j_event)
data4 = dd_frame.drop(index_rem)

month_file_start = start_date.strftime("%b")
day_file_start = start_date.strftime("%d")
year_file_start = start_date.year
month_file_end = end_date.strftime("%b")
day_file_end = end_date.strftime("%d")
year_file_end = end_date.year
xlsx_file = f"FF_{day_file_start}{month_file_start}{year_file_start}_{day_file_end}{month_file_end}{year_file_end}_NoOverlap.xlsx"
data4.to_excel(xlsx_file,index=False)